# Programming and Mathematics for AI
## Coursework

This notebook contains all solutions annotated via markdown for the Programming and Mathematics for AI coursework.

In [228]:
# Imports for the whole project
import math, random, time
import numpy as np
import pandas as pd
from matplotlib import pyplot
from keras.datasets import mnist
from collections import defaultdict
from keras.utils.np_utils import to_categorical

## Task 2
The second task is about classifying handwritten digits. We will use the MNIST dataset for training and testing.
The point of this task is to develop a multi-layer neural network for classification using mostly Numpy:
- Implement sigmoid and relu layers (with forward and backward pass)
- Implement a softmax output layer
- Implement a fully parameterizable neural network (number and types of layers, number of units)
- Implement an optimizer (e.g. SGD or Adam) and a stopping criterion of your choosing
- Train your Neural Network using backpropagation.

Evaluate different neural network architectures and compare your different results. You can also compare with the results presented in http://yann.lecun.com/exdb/mnist/

## Implementing the MNIST Dataset

Below I am splitting the MNIST dataset into training and testing data, as well as taking a small sample of the dataset to display the type of data we will be working with for visual and debugging purposes.

In [229]:
(trainX, trainY), (testX, testY) = mnist.load_data()

trainX = trainX.reshape(trainX.shape[0], trainX[0].shape[0] * trainX[0].shape[1])
testX = testX.reshape(testX.shape[0], testX[0].shape[0] * testX[0].shape[1])
trainY = to_categorical(trainY)
testY = to_categorical(testY)

## Class Definition

In [301]:
class MLNN:
    def __init__(self, iLayerSize=784, hLayerSizes=(784, 784), oLayerSize=10, learningRate=0.001, epochs=10):
        # Weights between hidden neurons and the output
        # These do not refer to neuron values, but the weights between them
        np.random.seed(100)
        self.parameters = {}
        self.learningRate = learningRate
        self.epochs = epochs

        self.layers = [iLayerSize]
        for hLayer in hLayerSizes:
            self.layers.append(hLayer)
        self.layers.append(oLayerSize)


        # Every input neuron connects to every first layer hidden neuron
        for i in range(1, len(self.layers)):
            self.parameters["W" + str(i)] = np.random.randn(self.layers[i - 1], self.layers[i]) * np.sqrt(1. / self.layers[i])
            self.parameters["B" + str(i)] = np.zeros((1, self.layers[i]))
            

    def forwardSigmoid(self, X):
        params = self.parameters
        params['A0'] = np.array(X)
        for i in range(1, len(self.layers)):
            params['Z' + str(i)] = np.dot(params['A' + str(i - 1)], params['W' + str(i)]) + params['B' + str(i)]
            if(i == len(self.layers) - 1):
                self.parameters['A' + str(i)] = self.softmax(self.parameters['Z' + str(i)])
            else:
                self.parameters['A' + str(i)] = self.sigmoid(self.parameters['Z' + str(i)])
        return params["A" + str(len(self.layers) - 1)]
    
    def backwardsSigmoid(self, Y, output):
        params = self.parameters
        changes = {}
        placeholder = self.parameters
        for i in reversed(range(1, len(self.layers))):
            if (i == len(self.layers) - 1):
                error = 2 * (output - Y) / output.shape[0] * self.softmax(params['Z' + str(i)], derivative=True)
                changes['W' + str(i)] = np.outer(error, params['A' + str(i - 1)])
            else:
                error = np.dot(error, params['W' + str(i + 1)].T) * self.sigmoid(params['Z' + str(i)], derivative=True)
                changes['W' + str(i)] = np.outer(error, params["A" + str(i - 1)])
        return changes
    
    def backwardsRelu(self, Y, output):
        params = self.parameters
        changes = {}
        placeholder = self.parameters
        for i in reversed(range(1, len(self.layers))):
            if (i == len(self.layers) - 1):
                error = 2 * (output - Y) / output.shape[0] * self.softmax(params['Z' + str(i)], derivative=True)
                changes['W' + str(i)] = np.outer(error, params['A' + str(i - 1)])
            else:
                error = np.dot(error, params['W' + str(i + 1)].T) * self.relu(params['Z' + str(i)], derivative=True)
                changes['W' + str(i)] = np.outer(error, params["A" + str(i - 1)])
        return changes
    
    def updateWeights(self, changes):
        for key, value in changes.items():
            self.parameters[key] -= self.learningRate * np.transpose(value)
    
    # To be used with training data
    def forwardRelu(self, X):
        params = self.parameters
        params['A0'] = np.array(X)
        for i in range(1, len(self.layers)):
            params['Z' + str(i)] = np.dot(params['A' + str(i - 1)], params['W' + str(i)]) + params['B' + str(i)]
            if(i == len(self.layers) - 1):
                self.parameters['A' + str(i)] = self.softmax(self.parameters['Z' + str(i)])
            else:
                self.parameters['A' + str(i)] = self.relu(self.parameters['Z' + str(i)])
        return params["A" + str(len(self.layers) - 1)]
    
    def accuracy(self, x_val, y_val):
        predictions = []
        for x, y in zip(x_val, y_val):
            output = self.forwardSigmoid(x)
            pred = np.argmax(output)
            predictions.append(pred == np.argmax(y))
        return np.mean(predictions)

    def trainSigmoid(self, x_train, y_train, x_test, y_test):
        start_time = time.time()
        for epoch in range(self.epochs):
            for x, y in zip(x_train, y_train):
                output = self.forwardSigmoid(x)
                changes = self.backwardsSigmoid(y, output)
                self.updateWeights(changes)
            accuracy = self.accuracy(x_test, y_test)
            print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
                epoch+1, time.time() - start_time, accuracy * 100
            ))
    
    def trainRelu(self, x_train, y_train, x_test, y_test):
        start_time = time.time()
        for epoch in range(self.epochs):
            for x, y in zip(x_train, y_train):
                output = self.forwardRelu(x)
                changes = self.backwardsRelu(y, output)
                self.updateWeights(changes)
            accuracy = self.accuracy(x_test, y_test)
            print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
                epoch+1, time.time() - start_time, accuracy * 100
            ))
    
    @staticmethod
    def sigmoid(x, derivative=False):
        if derivative:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        return 1/(1 + np.exp(-x))

    @staticmethod
    def softmax(x, derivative=False):
        # Numerically stable with large exponentials
        exps = np.exp(x - x.max())
        if derivative:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        return exps / np.sum(exps, axis=0)
    
    @staticmethod
    def relu(x, derivative=False):
        if derivative:
            x[x<=0] = 0
            x[x>0] = 1
            return x
        return np.maximum(0, x)

mlnn = MLNN(iLayerSize=len(trainX[0]), hLayerSizes=(128, 64), oLayerSize=10)
mlnn.trainRelu(trainX, trainY, testX[0], testY[0])

Epoch: 1, Time Spent: 89.48s, Accuracy: 100.00%
Epoch: 2, Time Spent: 171.68s, Accuracy: 100.00%
Epoch: 3, Time Spent: 258.67s, Accuracy: 100.00%
Epoch: 4, Time Spent: 347.39s, Accuracy: 100.00%
Epoch: 5, Time Spent: 434.20s, Accuracy: 100.00%
Epoch: 6, Time Spent: 520.95s, Accuracy: 100.00%
Epoch: 7, Time Spent: 605.08s, Accuracy: 100.00%
Epoch: 8, Time Spent: 688.42s, Accuracy: 100.00%
Epoch: 9, Time Spent: 772.17s, Accuracy: 100.00%
Epoch: 10, Time Spent: 854.35s, Accuracy: 100.00%
